In [1]:

# https://cookbook.openai.com/examples/structured_outputs_intro
# https://cookbook.openai.com/examples/structured_outputs_multi_agent

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
import json
from openai import OpenAI
client = OpenAI()

In [4]:
MODEL = "gpt-4o-2024-08-06"

### Text Summarization
In this example, we will ask the model to summarize articles following a specific schema.

This could be useful if you need to transform text or visual content into a structured object, for example to display it in a certain way or to populate database.

We will take web scraping as an example, using Wikipedia articles discussing inventions.

**Data preparation**. We will start by scraping content from multiple articles.

In [5]:
import requests
from bs4 import BeautifulSoup 

In [6]:
def get_article_content(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, "html.parser")
    html_content = soup.find("div", class_="mw-parser-output")
    content = "\n".join(p.text for p in html_content.find_all("p"))
    return content

In [7]:
urls = [
    # Article on CNNs
    "https://en.wikipedia.org/wiki/Convolutional_neural_network",
    # Article on LLMs
    "https://wikipedia.org/wiki/Large_language_model",
    # Article on MoE
    "https://en.wikipedia.org/wiki/Mixture_of_experts"
]

content = [get_article_content(url) for url in urls]

In [8]:
print(content)

['A convolutional neural network (CNN) is a regularized type of feed-forward neural network that learns features by itself via filter (or kernel) optimization. Vanishing gradients and exploding gradients, seen during backpropagation in earlier neural networks, are prevented by using regularized weights over fewer connections.[1][2] For example, for each neuron in the fully-connected layer, 10,000 weights would be required for processing an image sized 100 × 100 pixels. However, applying cascaded convolution (or cross-correlation) kernels,[3][4]  only 25 neurons are required to process 5x5-sized tiles.[5][6] Higher-layer features are extracted  from wider context windows, compared to lower-layer features.\n\nThey have applications in: \n\nCNNs are also known as shift invariant or space invariant artificial neural networks (SIANN), based on the shared-weight architecture of the convolution kernels or filters that slide along input features and provide translation-equivariant responses kn

In [10]:
from pydantic import BaseModel

summarization_prompt = '''
    You will be provided with content from an article about an invention.
    Your goal will be to summarize the article following the schema provided.
    Here is a description of the parameters:
    - invented_year: year in which the invention discussed in the article was invented
    - summary: one sentence summary of what the invention is
    - inventors: array of strings listing the inventor full names if present, otherwise just surname
    - concepts: array of key concepts related to the invention, each concept containing a title and a description
    - description: short description of the invention
'''

class ArticleSummary(BaseModel):
    invented_year: int
    summary: str
    inventors: list[str]
    description: str

    class Concept(BaseModel):
        title: str
        description: str

    concepts: list[Concept]

def get_article_summary(text: str):
    completion = client.beta.chat.completions.parse(
        model=MODEL,
        temperature=0.2,
        messages=[
            {"role": "system", "content": summarization_prompt},
            {"role": "user", "content": text}
        ],
        response_format=ArticleSummary,
    )

    return completion.choices[0].message.parsed

In [11]:
summaries = []

for i in range(len(content)):
    print(f"Analyzing article #{i+1}...")
    summaries.append(get_article_summary(content[i]))
    print("Done.")

Analyzing article #1...
Done.
Analyzing article #2...
Done.
Analyzing article #3...
Done.


In [12]:
def print_summary(summary):
    print(f"Invented year: {summary.invented_year}\n")
    print(f"Summary: {summary.summary}\n")
    print("Inventors:")
    for i in summary.inventors:
        print(f"- {i}")
    print("\nConcepts:")
    for c in summary.concepts:
        print(f"- {c.title}: {c.description}")
    print(f"\nDescription: {summary.description}")

In [14]:
for i in range(len(summaries)):
    print(f"ARTICLE {i}\n")
    print_summary(summaries[i])
    print("\n\n")

ARTICLE 0

Invented year: 1980

Summary: A convolutional neural network (CNN) is a type of artificial neural network designed to process data with a grid-like topology, such as images, by using convolutional layers to automatically learn spatial hierarchies of features.

Inventors:
- Fukushima

Concepts:
- Convolutional Layers: These layers apply a convolution operation to the input, passing the result to the next layer. They are designed to automatically learn spatial hierarchies of features from input images.
- Pooling Layers: Pooling layers reduce the dimensionality of each feature map but retain the most important information. This helps in reducing the computational load and controlling overfitting.
- ReLU Activation Function: The rectified linear unit (ReLU) is a non-linear activation function used in CNNs to introduce non-linearity into the model, allowing it to learn complex patterns.
- Weight Sharing: In CNNs, the same weights are used across different parts of the input, redu